# Analysis

In [ ]:
"""
_.py 
Analyses _
@author: Kevin Namink <k.w.namink@uu.nl>
Feedback and Comments by Sanli Faez
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import os

# To import our library with functions you might need to put 
# the DFSM.py file in the same folder as this file
import DFSM as dfsm


# Settings
# Usage: 
# Change folder, filename and framerate for the desired movie.
# It can be usefull to add some information to the info variable.

info = """
This is a place to put information on the data set 
from for example the readme file.
"""

# Configure:
folder = "/path/to/folder/"
filename = "00-filename_without_extension"
movienumber = 0
extension = ".hdf5"
framerate = 200.  # In frames per second

savedir = os.path.dirname(os.path.realpath("__file__"))+"/Analysis/"
dirName = savedir+filename+"_m"+str(movienumber)+"/"
if not os.path.exists(dirName):
    os.makedirs(dirName)

## Importing and handling data

Import and handle signal and camera data.

Usage: just run it, check dfsm for how it works.

In [ ]:
# Import DAC data:
DACdata = np.load(folder+filename+"_m"+str(movienumber)+".npy")
DAC_v, DAC_c, DAC_led = DACdata[:,0], DACdata[:,1], DACdata[:,2]

# Handle DAC data:
signal_frames, signal_offset, signal_amplitude, signal_shape = \
    dfsm.DAC_signal_properties(DAC_v)


# Imports camera data, while printing what movies are in the datafile:
data = dfsm.ImportHDF5data(folder+filename+extension)

# Find some properties of the movie: 
xrange, yrange, startframe, endframe, quicklook = \
    dfsm.movie_properties(data, movienumber)

# Plot quicklook, which is useful for estimating movie quality:
plt.plot(quicklook[startframe:endframe])
plt.title("Average of the center line")
plt.xlabel("Frame number")
plt.ylabel("Intensity (arb.u.)")
plt.show()


# Report found values:
Nperiods = (endframe-startframe)/signal_frames 
Tframe = 1/framerate
print("startframe: %d \nendframe: %d \nxrange: 0 to %d \nyrange: 0 to %d \nNperiods: %lf (%lf frames per signal in %d frames)" %(startframe, endframe, xrange, yrange, Nperiods, signal_frames, endframe-startframe))

# Create mean image and variance image while printing how much is filtered for the variance:
mean_img, var_img_MOD = \
    dfsm.get_image_mean_and_var(data, movienumber, startframe, endframe, maximumframes=5000)
if movienumber==0:
    usesamemeanimagetogetthesamepointsbecausethatiseasiertoprogram = mean_img

# Find offset between movie and signal
# Usage: run and check resulting alignment, manually overwrite if needed.

moviesignaloffset = dfsm.movie_signal_offset(quicklook, DAC_led)

# Overwrite manually if needed:
#moviesignaloffset = 6

# Print found value:
print("Found offset (movie - signal):", moviesignaloffset)

# Apply found offset by calculating movie and signal offsets:
mof, sof, msmax = \
    dfsm.create_offset_parameters(moviesignaloffset, quicklook, DAC_led)

# Plot result:
plt.figure(figsize=(8,4))
plt.subplot(121)
plt.plot(quicklook[0:60]/max(quicklook[mof:60-sof]), label="unedited movie")
plt.plot(DAC_led[0:60]/max(DAC_led[sof:60-mof]), label="unedited signal")
plt.title("alignment between signal and movie")
plt.legend()
plt.subplot(122)
plt.plot(quicklook[mof:60-sof]/max(quicklook[mof:60-sof]), label="edited movie")
plt.plot(DAC_led[sof:60-mof]/max(DAC_led[sof:60-mof]), label="edited signal")
plt.title("alignment between signal and movie")
plt.legend()
plt.show()


## Select a spot

Usage: 

Guess the center pixel of the spot you are interested in.
The found spot location is plotted and the found FWHM values are plotted.
This step can also indicate when particles are oversaturated or otherwise compromised.

What it does, 2 options:
1. It looks in an area 10 by 10 pixels around your guess for the peak intensity in the "mean_img" image. Then it finds the FWHM around this peak in the x and y direction. The rectangle found by these FWHM values around the maximum is used as the particle, so is summed over when looking for the particle intensity.
2. Look at a region around the guessed spot. (Activated by making do_a_region = True). The height and width variables then represent the amount of pixels to expand around the guess. It goes both up and down by the height variable. Same for width.


In [ ]:
# Select a spots (guess its center):
x_guess = 450
y_guess = 60

do_a_region = False
height  = 1
width   = 1




########################################
# Find a particle or set a region:

if not do_a_region:
    # Calculate local maximum intensity:
    x_found, y_found = \
        dfsm.find_max_I_around_guess(mean_img, x_guess, y_guess, areasize=10)
    # Find FWHM: 
    xx, x_FWHM, x1, yy, y_FWHM, y1 = \
        dfsm.find_FWHM_for_particle(mean_img, x_found, y_found, x_search=30, y_search=16)

else: 
    # Find center of region and find region:
    x_found = x_guess
    y_found = y_guess
    x1, x2, y1, y2 = dfsm.getROIaroundXYinFigure(x_found, y_found, mean_img, xsize=width, ysize=height)
    x_FWHM, y_FWHM = (x1, x2), (y1, y2)

    
########################################
# Plot some information:

if not do_a_region:
    # Plot mean_img and var_img_MOD with particle location (save it):
    plt.figure(figsize=(16,5))
    plt.imshow(np.log10(np.transpose(mean_img)))
    plt.scatter(x_guess, y_guess, s=10, color="xkcd:pink", label = "guess: I=%d"%mean_img[x_guess,y_guess])
    plt.scatter(x_found, y_found, s=10, color="xkcd:red", label = "found: I=%d"%mean_img[x_found,y_found])
    plt.title("Logarithm of Averaged Intensity")
    plt.xlabel("x") ; plt.ylabel("y") ; plt.legend()
    plt.show()
    
    plt.figure(figsize=(16,5))
    plt.imshow(np.log10(np.transpose(var_img_MOD)))
    plt.title("Logarithm of Variance") ; plt.xlabel("x") ; plt.ylabel("y")
    plt.scatter(x_guess, y_guess, s=10, color="xkcd:pink")
    plt.scatter(x_found, y_found, s=10, color="xkcd:red")
    plt.show()

    # Plot the found FWHM in relation to the data used to find it
    plt.figure(figsize=(18,6))
    
    plt.subplot(121)
    plt.plot(xx, '.', color='xkcd:baby blue', label='x direction')
    plt.scatter(x_FWHM[0]-x1, np.max(xx)/2, marker='>', color='xkcd:blue', label='x FWHM')
    plt.scatter(x_FWHM[1]-x1, np.max(xx)/2, marker='<', color='xkcd:blue')
    plt.plot(yy, '.', color='xkcd:baby pink', label='y direction')
    plt.scatter(y_FWHM[0]-y1, np.max(yy)/2, marker='>', color='xkcd:red', label='y FWHM')
    plt.scatter(y_FWHM[1]-y1, np.max(yy)/2, marker='<', color='xkcd:red')
    plt.title('FWHMaxima') ; plt.xlabel("distance (px)") ; plt.ylabel("intensity")
    plt.legend()
    
    plt.subplot(122)
    _, x2, _, y2 = dfsm.getROIaroundXYinFigure(x_found, y_found, mean_img, xsize=30, ysize=16)
    plt.imshow((mean_img[x1:x2,y1:y2]>mean_img[x_found, y_found]/2).T,cmap='Greys')
    plt.scatter(x_found-x1, y_found-y1, color='xkcd:white', label='Below half maximum')
    plt.scatter(x_found-x1, y_found-y1, color='xkcd:black', label='Above half maximum')
    plt.hlines(y_found-y1, x_FWHM[0]-x1,x_FWHM[1]-x1, color='xkcd:red')
    plt.vlines(x_found-x1, y_FWHM[0]-y1,y_FWHM[1]-y1, color='xkcd:red', label='FWHMaxima plotted found values')
    plt.scatter(x_found-x1, y_found-y1, color='xkcd:yellow', label='particle max I found location')
    plt.title('Information on particle size chosen') ; plt.legend()
    plt.xlabel("x distance (px)") ; plt.ylabel("y distance (px)")
    plt.show()

    print("x_found:", x_found, " and y_found:", y_found)
    

else:
    # Plot region (save it): 
    
    plt.figure(figsize=(16,5))
    plt.imshow(np.log10(np.transpose(mean_img)))    
    plt.vlines(x_FWHM[0], y_FWHM[0], y_FWHM[1], color='xkcd:red', label='region borders')
    plt.vlines(x_FWHM[1], y_FWHM[0], y_FWHM[1], color='xkcd:red')
    plt.hlines(y_FWHM[0], x_FWHM[0], x_FWHM[1], color='xkcd:red')
    plt.hlines(y_FWHM[1], x_FWHM[0], x_FWHM[1], color='xkcd:red')
    plt.scatter(x_found, y_found, s=10, color="xkcd:pink", label = "region center")
    plt.title("Logarithm of Averaged Intensity")
    plt.xlabel("x") ; plt.ylabel("y") ; plt.legend()
    plt.show()
    
    print("Area used around x_found:", x_found, " and y_found:", y_found)

## Process intensity data

Usage: 

Every step is commented on what it does, for more information: see dfsm

(Automatic saving is commented out)

In [ ]:
# Sum over particle adjusted for overlapping and skipping LED flash:
LEDskip = 20
particle = np.sum(np.sum(data[movienumber,1][x_FWHM[0]:x_FWHM[1], y_FWHM[0]:y_FWHM[1],LEDskip+mof:msmax-sof] - 96, axis=0), axis=0)

# Average out a (probably rounding) error that makes the data oscillate
# every 2 frames; in other words every other frame is offset by some value.
# Perhaps not always present
#particle = (np.roll(particle,1)+particle)/2

# Adjust DAC data for overlapping and skip the LED flash:
adjusted_signal = DAC_v[LEDskip+sof:msmax-mof]
adjusted_current = DAC_c[LEDskip+sof:msmax-mof]

# Correct particle drift by either fitting a polynominal or using a FFT filter
# options: polyorfft=['fft', 'poly'], polyorder=8 and fftfilter=16
# Notes: FFT usually works better but has some arguments against it that I, Kevin,
# do not personally really understand.
driftcorrected_particle = \
    dfsm.correct_particle_drift(particle, polyorfft='fft', polyorder=12, fftfilter=16)

fig = plt.figure(figsize=(12,6))
plt.plot(particle, color="xkcd:orange", label="signal before correction")
plt.plot(driftcorrected_particle, color='xkcd:green', label="signal after correction")

# Ratio of points allowed in a single cycle to be outside the whole measurements' gaussian 5% distance
outlier_rate = 0.1  
# Calculate cycle averages:
i, v, c = \
    dfsm.cycleaverage_particle_and_DAC(driftcorrected_particle, 
                                       adjusted_signal, adjusted_current, 
                                       signal_frames, signal_offset-LEDskip, 
                                       outlier_rate)
t_arr = Tframe*np.arange(len(i))

plt.title("Drift correction and accepted cycles", size=16)
plt.xlabel("frame", size=14)
plt.ylabel("intensity (counts)", size=14)
plt.legend()
#plt.savefig(dirName+"x%d_y%d_cycles.png"%(x_found, y_found))
plt.show()

# Plot processed intensity data:
# Usage: plots stuff, edit to your wish

##############
# Show intensity and potential:
fig = plt.figure(figsize=(10,4))

ax1 = fig.add_subplot(1,1,1)
color = 'tab:red'
ax1.set_xlabel('time ($s$)', size=16)
ax1.set_ylabel('$\mathrm{\Phi}_{\mathrm{cell}}$ ($V$)', size=16, color=color) 
ax1.plot(t_arr, v, '.', color=color)
ax1.tick_params('y', colors=color)

# Usage: 
# Every step is commented on what it does, for more information: see dfsm

ax11 = ax1.twinx() 
color = 'xkcd:royal blue'
ax11.set_ylabel('I/I$_{avr}$ ', size=16, color=color)
ax11.plot(t_arr, i/np.mean(i), '.', color=color)
ax11.tick_params('y', colors=color)

#plt.savefig(dirName+"x%d_y%d_intensity_and_potential.png"%(x_found, y_found))
plt.show()


###############
# Show intensity vs potential:
fig = plt.figure(figsize=(10,6))

# Apply a Savitzky-Golay filter to an array. This filter fits polynominals to
# part of the data to effectively average it. 
# First 3 arguments are resp. 
# 1. the to average data
# 2. the amount of datapoints to take
# 3. the order of the fitted polynominal
# deriv keyword is which order derivative should be reported
part_s = int(len(i)/10) + 1 - int(len(i)/10)%2  # get an always odd number that is approximately 1/10th of the size of the array
ia = savgol_filter(i, part_s, 3, mode='wrap', deriv=0)

# Alternatively use a Fourier filter with parameters:
# 1. the to filter data
# 2. the low frequencies to filter up to (should be 0)
# 3. the high frequencies to filter from 
#ia = dfsm.pixelFFTFilter(i, 0, 30)
#if len(ia) != len(i):  # Fix length issues
#    ia = np.append(ia, ia[0])

ax1 = fig.add_subplot(1,1,1)
ax1.set_xlabel('$\Phi_{cell}$ (V)', size=18)
ax1.set_ylabel('$I/I_{avr}$', size=18)

#ax1.plot(adjusted_signal, driftcorrected_particle/np.mean(i), '.', markersize=2, color='xkcd:grey') 

    
half = int(len(v)/2)

ax1.plot(v[:half], i[:half]/np.mean(i), '.', markersize=7, color='xkcd:royal blue')
ax1.plot(v[half:], i[half:]/np.mean(i), '.', markersize=7, color='xkcd:red')
ax1.plot(v[:half], ia[:half]/np.mean(i), '-', linewidth=4, color='xkcd:dark blue')
ax1.plot(v[half:], ia[half:]/np.mean(i), '-', linewidth=4, color='xkcd:dark red')
ax1.tick_params(labelsize=14)

#plt.savefig(dirName+"x%d_y%d_intensity_vs_potential.png"%(x_found, y_found))
plt.show()


npdatasave = np.array([v, i, adjusted_signal, driftcorrected_particle, x_FWHM, y_FWHM])

#savedirname = dirName+"x%d_y%d_data.npy"%(x_found, y_found)
#np.save(savedirname, npdatasave)

## Show current vs potential

In [ ]:
fig = plt.figure(figsize=(10,6))

ax1 = fig.add_subplot(1,1,1)
color = 'xkcd:dark green'
ax1.set_xlabel('$\Phi_{cell}$ (V)', size=18)
ax1.set_ylabel('$current$ (mA)', size=18)
ax1.plot(v, c/1000*1000, '.', markersize=7, color=color)
ax1.tick_params(labelsize=14)
plt.show()


###############
# Show averaged current vs potential:
fig = plt.figure(figsize=(10,6))

ca = savgol_filter(c, 51, 5, mode='wrap') 

ax1 = fig.add_subplot(1,1,1)
color = 'xkcd:dark green'
ax1.set_xlabel('$\Phi_{cell}$ (V)', size=18)
ax1.set_ylabel('$current$ (mA)', size=18)
ax1.plot(v, ca, '.', markersize=7, color=color)
ax1.tick_params(labelsize=14)
plt.show()